In [ ]:
# choose your saved model class
import model_scratch_v3 as m

import torch
import torch.nn.functional as F

In [ ]:
# load checkpoint corresponding to model
checkpoint = torch.load('model_scratch_v3.pt')


model = m.Net()
model.load_state_dict(checkpoint['state_dict'])
model.epoch = checkpoint['epoch']
model.min_validation_loss = checkpoint['min_validation_loss']
model.max_accuracy = checkpoint['max_accuracy']

In [ ]:
# Todo - handle all layers by being flexible about tensor dimensions
# Clean up logic as i only attached hook where needed (trash try block)

def printnorm(self, input, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
    class_name = self.__class__.__name__
    
    if class_name in ['Conv2d','Linear']:
        print('Inside ' + self.__class__.__name__ + ' forward')
        print('')
        print('input size:', input[0].size())
        print('output size:', output.data.size())
        print('output norm:', output.data.norm())

        try:
            output = F.relu(output)

            if output.dim() == 4:
                num_dead = (output.data == 0).sum(1).sum(1).sum(1)
            elif output.dim() == 2:
                num_dead = (output.data == 0).sum(1)
            else:
                print('unexpected dim of', output.dim)

            num_el = output.data.numel()
            print('% dead relus:', (num_dead.float()/num_el*100.).item())
            print('elements:', num_el)


        except:
            print('unexpeted error!')
    
    print('-'*100)


# model.conv2.register_forward_hook(printnorm)

# out = model(input)

In [ ]:
for param in model.modules():
    class_name = param.__class__.__name__
    print(class_name,end='')
    if class_name in ['Conv2d','Linear']:        
        param.register_forward_hook(printnorm)
        print(' << attaching hook')
    print('\n')

In [ ]:
x = torch.randn(1,3,64, 64)
output = model(x)
